In [22]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB

In [30]:
covid_data = pd.read_csv('covid_cleaned.csv')
covid_data = covid_data[covid_data['hosp_yn'] != 'Unknown']
covid_data = covid_data[covid_data['hosp_yn'] != 'Missing']
del covid_data['cdc_case_earliest_dt ']
encoded_data = covid_data.apply(LabelEncoder().fit_transform)
print(covid_data)

                    current_status     sex      age_group  \
0                    Probable Case  Female  30 - 39 Years   
1        Laboratory-confirmed case  Female  30 - 39 Years   
2                    Probable Case  Female  30 - 39 Years   
3        Laboratory-confirmed case  Female  30 - 39 Years   
4        Laboratory-confirmed case  Female  30 - 39 Years   
...                            ...     ...            ...   
1094546  Laboratory-confirmed case  Female  30 - 39 Years   
1094547              Probable Case  Female  30 - 39 Years   
1094548  Laboratory-confirmed case  Female  30 - 39 Years   
1094549  Laboratory-confirmed case  Female  30 - 39 Years   
1094550  Laboratory-confirmed case  Female  30 - 39 Years   

        race_ethnicity_combined hosp_yn icu_yn death_yn medcond_yn  
0           White, Non-Hispanic      No     No       No        Yes  
1           White, Non-Hispanic      No     No       No        Yes  
2           White, Non-Hispanic      No     No       No     

In [31]:
prior = covid_data.groupby('death_yn').size().div(len(covid_data)) 
print(prior)

death_yn
No     0.944622
Yes    0.055378
dtype: float64


In [32]:
likelihood = {}
likelihood['sex'] = covid_data.groupby(['death_yn', 'sex']).size().div(len(covid_data)).div(prior)
likelihood['race_ethnicity_combined'] = covid_data.groupby(['death_yn', 'race_ethnicity_combined']).size().div(len(covid_data)).div(prior)
likelihood['hosp_yn'] = covid_data.groupby(['death_yn', 'hosp_yn']).size().div(len(covid_data)).div(prior)
likelihood['icu_yn'] = covid_data.groupby(['death_yn', 'icu_yn']).size().div(len(covid_data)).div(prior)
likelihood['current_status'] = covid_data.groupby(['death_yn', 'current_status']).size().div(len(covid_data)).div(prior)
likelihood['medcond_yn'] = covid_data.groupby(['death_yn', 'medcond_yn']).size().div(len(covid_data)).div(prior)

print (likelihood)

{'sex': death_yn  sex   
No        Female    0.542005
          Male      0.457979
          Other     0.000016
Yes       Female    0.427200
          Male      0.572800
dtype: float64, 'race_ethnicity_combined': death_yn  race_ethnicity_combined                             
No        American Indian/Alaska Native, Non-Hispanic             0.004226
          Asian, Non-Hispanic                                     0.025735
          Black, Non-Hispanic                                     0.095445
          Hispanic/Latino                                         0.162691
          Multiple/Other, Non-Hispanic                            0.021310
          Native Hawaiian/Other Pacific Islander, Non-Hispanic    0.005643
          White, Non-Hispanic                                     0.684950
Yes       American Indian/Alaska Native, Non-Hispanic             0.008376
          Asian, Non-Hispanic                                     0.035047
          Black, Non-Hispanic                    

In [35]:
clf = MultinomialNB()
clf.fit(encoded_data.drop(['death_yn'], axis=1), encoded_data['death_yn'])


# {"Lab Confirmed": 'No', "Sex": 'Male',"Age":'20-29', "Race":"White", "Hospitalized":'no' , "ICU":'no', "Underlying Condition": 'no'}

X = np.array([0,1,2,6,0,0,0])
print (clf._joint_log_likelihood(X.reshape(1,-1)))
print ("Prediction of : ", clf.predict(X.reshape(1,-1)))



[[ -9.134292   -14.36668145]]
Prediction of :  [0]
